# Memory steering · Experiments walkthrough

A lightweight tour of `src/memory_steering/memory_steering_experiments.py`. We run the junction logistic and curvature-scaling helpers on small settings to inspect the resulting dimensionless collapses.


In [ ]:
import sys
from pathlib import Path


def _ensure_repo_on_path():
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for candidate in candidates:
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            sys.path.insert(0, str(candidate))
            return candidate
    return cwd

_ensure_repo_on_path()

from notebooks.utils import ensure_repo_path, allocate_artifacts, preview_json

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")



In [ ]:
try:
    from src.memory_steering.memory_steering_experiments import run_junction_logistic, run_curvature_scaling
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from src.memory_steering.memory_steering_experiments import run_junction_logistic, run_curvature_scaling

import numpy as np

X, P = run_junction_logistic(theta=1.8, delta_m_values=np.linspace(-1.5, 1.5, 9), trials=400)
print(preview_json({
    "delta_m_span": [float(X.min()), float(X.max())],
    "midpoint_probability": float(P[len(P)//2]),
}))

Xc, Yc = run_curvature_scaling(nx=21, ny=21, theta_values=(0.5, 1.5), pulses=48, mode="ray", dt=0.10, nsteps=160)
print(preview_json({
    "samples": len(Xc),
    "mean_kappa": float(np.nanmean(Yc)),
    "std_kappa": float(np.nanstd(Yc)),
}))


In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image, display

artifacts = allocate_artifacts("memory_steering", "experiments_notebook_demo")
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].plot(X, P, 'o-')
axes[0].set_xlabel('Theta * Delta m')
axes[0].set_ylabel('P(A)')
axes[0].set_title('Junction logistic collapse')
axes[1].plot(Xc, Yc, 'o', alpha=0.8)
axes[1].set_xlabel('Theta * |grad m|')
axes[1].set_ylabel('mean(kappa_path)')
axes[1].set_title('Curvature scaling sample')
for ax in axes:
    ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(artifacts['figure'], dpi=160)
plt.close(fig)
print(preview_json({name: str(path) for name, path in artifacts.items()}))
display(Image(filename=str(artifacts['figure'])))


## Notebook checklist

* The reduced trials keep runtime low while still illustrating the logistic collapse.
* Curvature scaling uses the ray-limit integrator; inspect the scatter or reuse the returned arrays for plotting.
* Artifact paths mirror the production harness should you export figures/logs directly from the notebook.
* Expand the pulse counts when validating against the paper-quality acceptance thresholds.
